# Advanced guide for data modeling based cogshop runs

**This tutorial covers more advanced techniques than quickstart and will not go into details covered there, such as setting up the Powerops Client**


## Preparation


NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file or separate files).


In [1]:
# You can control which configuration files are used.
# In this case, the configuration files are located two levels above, in the root of the repository.
from dotenv import load_dotenv
from pathlib import Path
from cognite.powerops import PowerOpsClient
from cognite.powerops._version import __version__

# Adjust the path to the configs file if needed
root = Path().resolve().parent

load_dotenv(dotenv_path=str(root / ".env"))
powerops = PowerOpsClient.from_config(str(root /"power_ops_config.yaml"))

print(__version__)  # Print the version of the package

0.111.0


In [2]:
def upload_file(local_file_name: str, external_id: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / local_file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=local_file_name,
        data_set_id=powerops.datasets.write_dataset_id,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id

## Example 1: Use multiple files for shop files
Very similar to the example in quickstart, but using multiple files for `ShopCase`.

1. Upload the files that will be loaded into shop, keeping their external ids
2. Set up a `ShopCase`, using the SDK
3. Write the `ShopCase` instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that `ShopCase`
5. List the `ShopResult` referencing the `ShopCase` 


### Step 1: Upload a files 

In [3]:
# NB! The case in this example is minimal and just for demo purposes.
example_case_fbu_1 = upload_file(
    local_file_name="b_example_fornebu_without_commands.yaml",
    external_id="example_case_fornebu",
)
example_case_fbu_2 = upload_file(local_file_name="b_example_commands.yaml", external_id="example_commands")

print(f" File reference 1: '{example_case_fbu_1}', File reference 2: '{example_case_fbu_2}'")

 File reference 1: 'example_case_fornebu', File reference 2: 'example_commands'


### Step 2: Preparing a ShopCase

The main difference here is that `shop_file_list` will contain more than one tuple.

Unlike the quickstart, in this example we will let `prepare_shop_case` generate all the external IDs.

We will also let the file reference double as the name of the file. 


In [4]:
import datetime

#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]


shop_case_1 = powerops.cogshop.prepare_shop_case(
    shop_file_list=shop_file_list,
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    model_name="my_model",
    scenario_name="my_scenario",
)
shop_case_1

,value
space,power_ops_instances
external_id,shopcase:fbaead0a55884c22b84999a407f4c294
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [8]:
# We can also look at just the shop files
for file in shop_case_1.shop_files:
    print(
        f"File name: '{file.name}', File reference: '{file.file_reference}', Data modeling external id: '{file.external_id}'"
    )

File name: 'example_case_fornebu', File reference: 'example_case_fornebu', Data modeling external id: 'shopfile:51bf20345d6f4509b81b8808bd59db9d'
File name: 'example_commands', File reference: 'example_commands', Data modeling external id: 'shopfile:2b334bc02cfc4c0781d1c0387a3d2cb6'


### Step 3: Upload the ShopCase

In [9]:
powerops.v1.upsert(shop_case_1)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:fbaead0a55884c22b84999a407f4c294', version=1) at 0x7f90e85eff50>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:2e8c8e6981f749b08bf210304a677e04', version=1) at 0x7f9096f27210>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:e03514e4906744c4ab44b348afb9dbaf', version=1) at 0x7f9094eb2bd0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:51bf20345d6f4509b81b8808bd59db9d', version=1) at 0x7f9094eb2810>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:2b334bc02cfc4c0781d1c0387a3d2cb6', version=1) at 0x7f9094eb3410>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='shopcase:fbaead0a55884c22b84999a407f4c294:shopfile:51bf20345d6f4509b81b8808bd59db9d', version=1) at 0x7f9094eb3f10>, <EdgeApplyResult(space='power_ops_instances', external_id='shopcase:fbaead0a55884c22b84999a407f4c294:shopfile:2b3

In [10]:
from cognite.powerops.client._generated.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(shop_case_1.external_id)
retrieved_shop_case

,value
space,power_ops_instances
external_id,shopcase:fbaead0a55884c22b84999a407f4c294
data_record,"{'version': 1, 'last_updated_time': 2025-01-30..."
node_type,None
scenario,shopscenario:2e8c8e6981f749b08bf210304a677e04
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,None


*It is known that the `retrive` endpoint can be unstable*. 
We have a graphQL fallback for this case.  This is a less efficient way to retrieve the shop case, but it is useful in case the `retrive` fails

See details in the docstring of `retrieve_shop_case_graphql`

In [11]:
graph_ql_shop_case = powerops.cogshop.retrieve_shop_case_graphql(shop_case_1.external_id)
graph_ql_shop_case

,value
space,power_ops_instances
external_id,shopcase:fbaead0a55884c22b84999a407f4c294
data_record,"{'version': 0, 'last_updated_time': 2025-01-30..."
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [12]:
powerops.cogshop.trigger_shop_case(shop_case_1.external_id)

### Step 5: View the ShopResult generated 

It may take a while to run (Cog)Shop.

A new things to note:
* If the run is not completed then no results are returned. 
* If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.
* Technically it is possible to run `trigger_shop_case` multiple times for the same case. For this reason we suggest using lists to view `ShopResult`s. 
  

Explanations for (some) of the fields of the `ShopResult`:

* External id: The external id of the `ShopResult`
* Case: The external id of the `ShopCase` that the instance belongs to
* Objective value: The objective value of the Shop execution
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains the keys `last_updated_time` and `created_time`. These can be used to differentiate results from the same `ShopCase`



#### 1. Using list

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned

In [16]:
from cognite.powerops.client._generated.data_classes import ShopResultList

result_list_1: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_1.external_id
)  # This can easily be converted to a pandas DataFrame

print(f"There is/are {len(result_list_1)} result(s) for the case with external id '{shop_case_1.external_id}'")

There is/are 1 result(s) for the case with external id 'shopcase:fbaead0a55884c22b84999a407f4c294'


**Graph QL list?**

As is possible with all cognite data models, it is possible to render a Graph QL query to list the results. Follow the example in `retrieve_shop_case_graphql` function or create your own in fusion.

#### 2. Retrieve using the external ID  

In [18]:
from cognite.powerops.client._generated.data_classes import ShopResult

shop_result_1: ShopResult = powerops.cogshop.retrieve_shop_result(result_list_1[0].external_id)
shop_result_1

,value
space,power_ops_instances
external_id,shop_result__db74590d-2c0a-47b4-8fda-282708a59d2d
data_record,"{'version': 1, 'last_updated_time': 2025-01-30..."
node_type,None
case,shopcase:fbaead0a55884c22b84999a407f4c294
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-db74590d-2c0a-47b4-8fda-...
post_run,POWEROPS_SHOP_post-run-db74590d-2c0a-47b4-8fda...
messages,POWEROPS_SHOP_shop-db74590d-2c0a-47b4-8fda-282...
cplex_logs,POWEROPS_SHOP_cplex-db74590d-2c0a-47b4-8fda-28...


In [19]:
# Downloading one of the files to tmp directory and printing the first 10 lines
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "log_file.log")
    powerops.cdf.files.download_to_path(external_id=shop_result_1.messages, path=pre_run_file_path)
    with open(pre_run_file_path) as log_file:
        print("".join(log_file.readlines()[:10]))

INFORMATION: 1032
15.6.1.0 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-05-10

INFORMATION: 1047
Current time: Thu Jan 30 13:43:28 2025

DIAGNOSIS WARNING: 3250
Reservoir Dalbysvatn: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Dalbysvatn: 0 errors 1 warnings.



### Step 4 (optional): Prepare to delete created resources

In [20]:
all_external_ids_1 = [
    shop_case_1.scenario.external_id,
    shop_case_1.scenario.model.external_id,
    shop_case_1.external_id,
    shop_case_1.shop_files[0].external_id,
    shop_case_1.shop_files[1].external_id,
    shop_result_1.external_id,
]

all_external_ids_1  # Record these for deletion

['shopscenario:2e8c8e6981f749b08bf210304a677e04',
 'shopmodel:e03514e4906744c4ab44b348afb9dbaf',
 'shopcase:fbaead0a55884c22b84999a407f4c294',
 'shopfile:51bf20345d6f4509b81b8808bd59db9d',
 'shopfile:2b334bc02cfc4c0781d1c0387a3d2cb6',
 'shop_result__db74590d-2c0a-47b4-8fda-282708a59d2d']

## Example 2: Reusing a ShopScenario instance
As mentioned in the intro Quickstart, the recommend way to run CogShop is by having a static model and use scenarios to vary configurations. 

However, if we know that we want to run several `ShopCases` using the same configurations, we can reuse a `ShopScenario`/`ShopModel` instead of creating several identical instances.


1. Create a `ShopScenario` and `ShopModel` that set the desired configuration (here `shop_version`). We will create two for demo purposes using the data classes directly 
2. Write the `ShopScenario` to CDF. 
3. Set up a `ShopCase`, preparing shop file list with names and references as explained in Example 1. Showing two ways to reference `ShopScenario`
4. Write the cases to  to CDF. 
4. Trigger shop executions of the `ShopCase`a  and list the `ShopResult` as in previous examples 

As in the previous example, we will be letting the SDK generate all external id. Again they can be 

### Step 1: Create reusable ShopScenarios (with ShopScenarios)

`ShopScenario`s and `ShopModel`s created this way, will have several unused fields that are `None`. This is due to the there usage here being very different from their intended usage.   


See the Power Ops SDK or the Data Models in Fusion for more information of what the fields represent

In [24]:
# Not necessary hte only versions available, but we can list available files Shop Version in CDF.
powerops.cogshop.list_shop_versions()

['SHOP-15.4.1.1-pyshop-python3.11-linux.zip',
 'SHOP-16.0.3-pyshop-python3.11-linux.zip',
 'SHOP-16.0.2-pyshop-python3.11-linux.zip',
 'SHOP-15.6.1.0-pyshop-python3.11-linux.zip']

In [25]:
from cognite.powerops.client._generated.data_classes import ShopModelWrite, ShopScenarioWrite

# Using shop version 15.6.1.0
reusable_model_15_6_1_0 = ShopModelWrite(
    name="reusable_shop_model_15_6_1_0",
    shop_version="15.6.1.0",
    model_version="1.0",
    model=None,
    penalty_limit=None,
    cog_shop_version=None,
    base_attribute_mappings=None,
)

reusable_scenario_15_6_1_0 = ShopScenarioWrite(
    name="reusable_shop_scenario_15_6_1_0",
    model=reusable_model_15_6_1_0,
    commands=None,
    source=None,
    output_definition=None,
    attribute_mappings_override=None,
)

# Using shop version 16.0.3
reusable_model_16_0_3 = ShopModelWrite(
    name="reusable_shop_model_16_0_3",
    shop_version="16.0.3",
    model_version="1.0",
    model=None,
    penalty_limit=None,
    cog_shop_version=None,
    base_attribute_mappings=None,
)

reusable_scenario_16_0_3 = ShopScenarioWrite(
    name="reusable_shop_scenario_16_0_3",
    model=reusable_model_16_0_3,
    commands=None,
    source=None,
    output_definition=None,
    attribute_mappings_override=None,
)

In [26]:
# Looking at the models
print(f"Shop Model 15.5.1.0 '{reusable_model_15_6_1_0.external_id}'")
print(f"Shop Model 16.0.3   '{reusable_model_16_0_3.external_id}'")

Shop Model 15.5.1.0 'shopmodel:c3151fd33aee42e880a298dfe5b99275'
Shop Model 16.0.3   'shopmodel:cb5dc778fcf24f528451673f3d32cb0d'


In [27]:
# Looking at the scenarios
print(f"Shop Scenario 15.6.1.0: '{reusable_scenario_15_6_1_0.external_id}'")
print(f"Shop Scenario 16.0.3:   '{reusable_scenario_16_0_3.external_id}'")

Shop Scenario 15.6.1.0: 'shopscenario:01426be5d6ef4100a03c0ca6fa503076'
Shop Scenario 16.0.3:   'shopscenario:da73790217944684b5d103a972afaa1d'


### Step 2: Write the scenarios to CDF

Since the Scenarios are referencing `Write` instances of the `ShopModel`s, just upserting the scenarios is sufficient. 
Here we will only upsert 1 of the scenarios to demonstrate two different ways to prepare a case with an existing scenario.

In [28]:
powerops.v1.upsert(reusable_scenario_15_6_1_0)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopscenario:01426be5d6ef4100a03c0ca6fa503076', version=1) at 0x7f908f05cd50>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:c3151fd33aee42e880a298dfe5b99275', version=1) at 0x7f908f05e710>], edges=[], time_series=[], files=[], sequences=[])

### Step 3: Create new Shop Cases, referencing the newly created scenarios

We will be reusing the files from the example above

In [29]:
#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]

In [30]:
shop_case_15_6_1_0 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_15_6_1_0.external_id,  # Note that we are using the external id of the scenario
)
shop_case_15_6_1_0

,value
space,power_ops_instances
external_id,shopcase:e445c110f3b94fcf8a2e2a4bdce6d244
data_record,{'existing_version': None}
node_type,None
scenario,shopscenario:01426be5d6ef4100a03c0ca6fa503076
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [31]:
# This will fail because the scenario does not exist in CDF yet (we did not upsert it)
will_fail_shop_case_16_0_3 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_16_0_3.external_id,  # Note that this is the external id of the scenario
)

ValueError: Invalid shop scenario reference: shopscenario:da73790217944684b5d103a972afaa1d.

In [32]:
# However, if we reference the Write object in the case creation it will be work.
# This is because a chained series of Write obejcts (which `prepare_shop_case_with_existing_scenario` returns) can be upserted at the same time.
shop_case_16_0_3 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_16_0_3,  # Note that this is the Write object
)
shop_case_16_0_3

,value
space,power_ops_instances
external_id,shopcase:3891c293108345469bff404960eaf0a7
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Upserting the new cases. 

Notice that the resources are bundled together as a list when we write more than one resource 

In [33]:
powerops.v1.upsert([shop_case_15_6_1_0, shop_case_16_0_3])

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:e445c110f3b94fcf8a2e2a4bdce6d244', version=1) at 0x7f9094e8f7d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:0755f2f7292c4859ba8bd563bc4d95f1', version=1) at 0x7f908f05d4d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:665d941cc9c548e6a781e4ba1184c369', version=1) at 0x7f908e701b90>, <NodeApplyResult(space='power_ops_instances', external_id='shopcase:3891c293108345469bff404960eaf0a7', version=1) at 0x7f908e701b50>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:da73790217944684b5d103a972afaa1d', version=1) at 0x7f908e701010>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:cb5dc778fcf24f528451673f3d32cb0d', version=1) at 0x7f908e700a10>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:6c1c37e13042422595fe8d65ead2f1f3', version=1) at 0x7f908e700d90>, <NodeApplyResult(space='power

### Step 5: Trigger Shop Executions and inspect results

In [34]:
powerops.cogshop.trigger_shop_case(shop_case_15_6_1_0.external_id)
powerops.cogshop.trigger_shop_case(shop_case_16_0_3.external_id)


Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned. 
If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.


In [36]:
result_list_15_5_1_0: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_15_6_1_0.external_id
)
print(f"There is/are {len(result_list_15_5_1_0)} result(s) for the case '{shop_case_15_6_1_0.external_id}'")

There is/are 1 result(s) for the case 'shopcase:e445c110f3b94fcf8a2e2a4bdce6d244'


In [37]:
shop_result_15_6_1_0: ShopResult = powerops.cogshop.retrieve_shop_result(result_list_15_5_1_0[0].external_id)
shop_result_15_6_1_0

,value
space,power_ops_instances
external_id,shop_result__0b84d204-349d-4bca-9d0f-d0acad597548
data_record,"{'version': 1, 'last_updated_time': 2025-01-30..."
node_type,None
case,shopcase:e445c110f3b94fcf8a2e2a4bdce6d244
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-0b84d204-349d-4bca-9d0f-...
post_run,POWEROPS_SHOP_post-run-0b84d204-349d-4bca-9d0f...
messages,POWEROPS_SHOP_shop-0b84d204-349d-4bca-9d0f-d0a...
cplex_logs,POWEROPS_SHOP_cplex-0b84d204-349d-4bca-9d0f-d0...


In [38]:
result_list_16_0_3: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_16_0_3.external_id, limit=1
)
print(f"There is/are {len(result_list_16_0_3)} result(s) for the case '{shop_case_16_0_3.external_id}'")

There is/are 1 result(s) for the case 'shopcase:3891c293108345469bff404960eaf0a7'


In [39]:
shop_result_16_0_3: ShopResult = powerops.cogshop.retrieve_shop_result(result_list_16_0_3[0].external_id)
shop_result_16_0_3

,value
space,power_ops_instances
external_id,shop_result__10dcf41a-661b-4c53-95a9-19fceed3ede7
data_record,"{'version': 1, 'last_updated_time': 2025-01-30..."
node_type,None
case,shopcase:3891c293108345469bff404960eaf0a7
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-10dcf41a-661b-4c53-95a9-...
post_run,POWEROPS_SHOP_post-run-10dcf41a-661b-4c53-95a9...
messages,POWEROPS_SHOP_shop-10dcf41a-661b-4c53-95a9-19f...
cplex_logs,POWEROPS_SHOP_cplex-10dcf41a-661b-4c53-95a9-19...


Looking at the `pre_run`s we can see that the cases/results generated were using different versions of SHOP, while the model is the the same one.

In [40]:
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "pre_run.yaml")
    powerops.cdf.files.download_to_path(external_id=shop_result_15_6_1_0.pre_run, path=pre_run_file_path)
    with open(pre_run_file_path) as log_file:
        print("".join(log_file.readlines()[:15]))

print("==========================================\n")

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "pre_run.yaml")
    powerops.cdf.files.download_to_path(external_id=shop_result_16_0_3.pre_run, path=pre_run_file_path)
    with open(pre_run_file_path) as log_file:
        print("".join(log_file.readlines()[:15]))

time:
  starttime: 2023-01-19 23:00:00
  endtime: 2023-01-29 23:00:00
  timeunit: minute
  timeresolution:
    2023-01-19 23:00:00: 60
    2023-01-22 23:00:00: 240
model:
  global_settings:
    global_settings:
      shop_version: 15.6.1.0 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-05-10
  creek_intake:
    Blakstad:
      net_head: 641
      max_inflow: 12


time:
  starttime: 2023-01-19 23:00:00
  endtime: 2023-01-29 23:00:00
  timeunit: minute
  timeresolution:
    2023-01-19 23:00:00: 60
    2023-01-22 23:00:00: 240
model:
  global_settings:
    global_settings:
      shop_version: 16.0.3 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-08-16
  creek_intake:
    Blakstad:
      net_head: 641
      max_inflow: 12



### Step 5 (optional): Prepare to delete created resources

In [41]:
all_external_ids_2 = [
    reusable_scenario_15_6_1_0.external_id,
    reusable_scenario_15_6_1_0.model.external_id,
    reusable_scenario_16_0_3.external_id,
    reusable_scenario_16_0_3.model.external_id,
    shop_case_15_6_1_0.external_id,
    shop_case_15_6_1_0.shop_files[0].external_id,
    shop_case_15_6_1_0.shop_files[1].external_id,
    shop_case_16_0_3.external_id,
    shop_case_16_0_3.shop_files[0].external_id,
    shop_case_16_0_3.shop_files[1].external_id,
    shop_result_15_6_1_0.external_id,
    shop_result_16_0_3.external_id,
]
all_external_ids_2  # Record these for deletion

['shopscenario:01426be5d6ef4100a03c0ca6fa503076',
 'shopmodel:c3151fd33aee42e880a298dfe5b99275',
 'shopscenario:da73790217944684b5d103a972afaa1d',
 'shopmodel:cb5dc778fcf24f528451673f3d32cb0d',
 'shopcase:e445c110f3b94fcf8a2e2a4bdce6d244',
 'shopfile:0755f2f7292c4859ba8bd563bc4d95f1',
 'shopfile:665d941cc9c548e6a781e4ba1184c369',
 'shopcase:3891c293108345469bff404960eaf0a7',
 'shopfile:6c1c37e13042422595fe8d65ead2f1f3',
 'shopfile:6fc77d53f2d8499eb8a1c391efbc6c58',
 'shop_result__0b84d204-349d-4bca-9d0f-d0acad597548',
 'shop_result__10dcf41a-661b-4c53-95a9-19fceed3ede7']

## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [42]:
all_external_ids = all_external_ids_1 + all_external_ids_2
all_external_ids

['shopscenario:2e8c8e6981f749b08bf210304a677e04',
 'shopmodel:e03514e4906744c4ab44b348afb9dbaf',
 'shopcase:fbaead0a55884c22b84999a407f4c294',
 'shopfile:51bf20345d6f4509b81b8808bd59db9d',
 'shopfile:2b334bc02cfc4c0781d1c0387a3d2cb6',
 'shop_result__db74590d-2c0a-47b4-8fda-282708a59d2d',
 'shopscenario:01426be5d6ef4100a03c0ca6fa503076',
 'shopmodel:c3151fd33aee42e880a298dfe5b99275',
 'shopscenario:da73790217944684b5d103a972afaa1d',
 'shopmodel:cb5dc778fcf24f528451673f3d32cb0d',
 'shopcase:e445c110f3b94fcf8a2e2a4bdce6d244',
 'shopfile:0755f2f7292c4859ba8bd563bc4d95f1',
 'shopfile:665d941cc9c548e6a781e4ba1184c369',
 'shopcase:3891c293108345469bff404960eaf0a7',
 'shopfile:6c1c37e13042422595fe8d65ead2f1f3',
 'shopfile:6fc77d53f2d8499eb8a1c391efbc6c58',
 'shop_result__0b84d204-349d-4bca-9d0f-d0acad597548',
 'shop_result__10dcf41a-661b-4c53-95a9-19fceed3ede7']

In [43]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shopscenario:2e8c8e6981f749b08bf210304a677e04'), NodeId(space='power_ops_instances', external_id='shopmodel:e03514e4906744c4ab44b348afb9dbaf'), NodeId(space='power_ops_instances', external_id='shopcase:fbaead0a55884c22b84999a407f4c294'), NodeId(space='power_ops_instances', external_id='shopfile:51bf20345d6f4509b81b8808bd59db9d'), NodeId(space='power_ops_instances', external_id='shopfile:2b334bc02cfc4c0781d1c0387a3d2cb6'), NodeId(space='power_ops_instances', external_id='shop_result__db74590d-2c0a-47b4-8fda-282708a59d2d'), NodeId(space='power_ops_instances', external_id='shopscenario:01426be5d6ef4100a03c0ca6fa503076'), NodeId(space='power_ops_instances', external_id='shopmodel:c3151fd33aee42e880a298dfe5b99275'), NodeId(space='power_ops_instances', external_id='shopscenario:da73790217944684b5d103a972afaa1d'), NodeId(space='power_ops_instances', external_id='shopmodel:cb5dc778fcf24f528451673f3d32cb0d'), NodeId(s